## Slack to GPT

터미널에서 해당 명령어 실행해서 크롬 원격 디버그창을 띄워놓은 상태에서 코드 실행

[windows]

start chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\temp"

[MacOS]

sudo /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222

In [1]:
%pip install selenium
%pip install webdriver-manager


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.chrome.options import Options
import time

팀원LeeSH

1. 기능을 함수로 쪼개서 함수별로 주석이 있으면 좋겠습니다

2. while 문 진입 후 finde_element 가 없을경우 exception이 던져지기 때문에 try-catch가 있으면 좋을것 같아요

In [ ]:
your_email = "rudah69@gmail.com"

# 원격 디버그 크롬으로 접속하기 위한 옵션값입니다
options = Options()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

driver_slack = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_gpt = webdriver.Chrome(options=options)

driver_slack.get("https://slack.com/signin#/signin")

driver_gpt.get("https://chat.openai.com/")
driver_gpt.implicitly_wait(10)


id_input = driver_slack.find_element(By.ID, "signup_email")
actions = ActionChains(driver_slack)
actions.send_keys_to_element(id_input, your_email).perform()
actions.send_keys(Keys.RETURN).perform()

input("Press Enter to continue...") 
## Chat GPT의 경우에는 구글 로그인같은 내역을 남겨놔야 접속이 되므로 완전 자동화는 힘들다.
## 조건 1. driver_slack으로 생성된 창을 로그인하여 웹버전 slack에서 메세지를 읽고 보낼 채널까지 들어간다.
## 조건 2. 원격 디버그 chrome에서 Chat gpt 홈페이지에 접속한 뒤, 개발자 도구를 열어서 화면 좌측의 사이드바를 없앤다.
## 해당 조건을 충족시킨 뒤에 엔터를 입력하여 다음 단계로 넘어간다.

slack_recent_message = ""
slack_to_gpt_input = ""
recent_answer = ""

while True:
    chats = driver_slack.find_elements(By.CLASS_NAME, "p-rich_text_section")

    if chats[-1].text == slack_recent_message:
        # 최근 메세지가 이전에 받았던 메세지와 동일할 경우 스킵
        print("슬랙 메세지가 업데이트 되지 않았습니다.")
        time.sleep(5)
        continue
    elif chats[-1].text in recent_answer:
        # 자문 자답 방지용, 대화 내용이 최근 답변과 유사할 경우 스킵
        print("슬랙 메세지가 업데이트 되지 않았습니다.")
        time.sleep(5)
        continue
    
    slack_recent_message = chats[-1].text

    gpt_input = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[2]/form/div/div[1]/textarea')
    gpt_button = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[2]/form/div/div[1]/button')

    actions_gpt = ActionChains(driver_gpt)
    actions_gpt.send_keys_to_element(gpt_input, slack_recent_message).perform()
    actions_gpt.click(gpt_button).perform()
    
    driver_gpt.implicitly_wait(3)
    main_divs = driver_gpt.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div')
    answer_num = len(main_divs) - 1
    
    #Chat GPT가 대답하는 과정이 길기 때문에 대답 하는 동안에는 체크하다가 대답이 끝나면 전송
    delay_check = 0 

    recent_answer = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div[%s]/div/div[2]/div[1]/div/div' % answer_num).text
    while delay_check <= 2:
        time.sleep(0.5)
        tmp_answer = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div[%s]/div/div[2]/div[1]/div/div' % answer_num).text
        if recent_answer == tmp_answer:
            #저장된 대답과 지금 받은 대답이 같을 경우 delay_check를 증가시킨다.
            delay_check += 1
        else:
            #저장된 대답과 지금 받은 대답이 다를 경우 delay_check를 초기화하고 지금 받은 대답을 저장된 대답으로 교체한다
            delay_check = 0
            recent_answer = tmp_answer
    
    # GPT가 대답한 메세지를 slack으로 전송
    slack_input = driver_slack.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div[5]/div[2]/div/div[4]/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[1]/p')
    actions.send_keys_to_element(slack_input, recent_answer).perform()
    actions.send_keys(Keys.RETURN).perform()

## Gathertown to GPT

게더타운에서 NPC 역할을 하도록 코드를 수정해보면 어떨까?

In [ ]:
your_email = "rudah69@gmail.com"

# 원격 디버그 크롬으로 접속하기 위한 옵션값입니다
options = Options()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

driver_town = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_gpt = webdriver.Chrome(options=options)

driver_town.get("https://slack.com/signin#/signin")

driver_gpt.get("https://chat.openai.com/")
driver_gpt.implicitly_wait(10)


id_input = driver_town.find_element(By.ID, "signup_email")
actions = ActionChains(driver_town)
actions.send_keys_to_element(id_input, your_email).perform()
actions.send_keys(Keys.RETURN).perform()

input("Press Enter to continue...") 
## Chat GPT의 경우에는 구글 로그인같은 내역을 남겨놔야 접속이 되므로 완전 자동화는 힘들다.
## 조건 1. driver_slack으로 생성된 창을 로그인하여 웹버전 slack에서 메세지를 읽고 보낼 채널까지 들어간다.
## 조건 2. 원격 디버그 chrome에서 Chat gpt 홈페이지에 접속한 뒤, 개발자 도구를 열어서 화면 좌측의 사이드바를 없앤다.
## 해당 조건을 충족시킨 뒤에 엔터를 입력하여 다음 단계로 넘어간다.

town_recent_message = ""
town_to_gpt_input = ""
recent_answer = ""

while True:
    chats = driver_town.find_elements(By.CLASS_NAME, "p-rich_text_section")

    if chats[-1].text == town_recent_message:
        # 최근 메세지가 이전에 받았던 메세지와 동일할 경우 스킵
        print("슬랙 메세지가 업데이트 되지 않았습니다.")
        time.sleep(5)
        continue
    elif chats[-1].text in recent_answer:
        # 자문 자답 방지용, 대화 내용이 최근 답변과 유사할 경우 스킵
        print("슬랙 메세지가 업데이트 되지 않았습니다.")
        time.sleep(5)
        continue
    
    town_recent_message = chats[-1].text

    gpt_input = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[2]/form/div/div[1]/textarea')
    gpt_button = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[2]/form/div/div[1]/button')

    actions_gpt = ActionChains(driver_gpt)
    actions_gpt.send_keys_to_element(gpt_input, town_recent_message).perform()
    actions_gpt.click(gpt_button).perform()
    
    driver_gpt.implicitly_wait(3)
    main_divs = driver_gpt.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div')
    answer_num = len(main_divs) - 1
    
    #Chat GPT가 대답하는 과정이 길기 때문에 대답 하는 동안에는 체크하다가 대답이 끝나면 전송
    delay_check = 0 

    recent_answer = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div[%s]/div/div[2]/div[1]/div/div' % answer_num).text
    while delay_check <= 2:
        time.sleep(0.5)
        tmp_answer = driver_gpt.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div[%s]/div/div[2]/div[1]/div/div' % answer_num).text
        if recent_answer == tmp_answer:
            #저장된 대답과 지금 받은 대답이 같을 경우 delay_check를 증가시킨다.
            delay_check += 1
        else:
            #저장된 대답과 지금 받은 대답이 다를 경우 delay_check를 초기화하고 지금 받은 대답을 저장된 대답으로 교체한다
            delay_check = 0
            recent_answer = tmp_answer
    
    # GPT가 대답한 메세지를 slack으로 전송
    town_input = driver_town.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div[5]/div[2]/div/div[4]/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[1]/p')
    actions.send_keys_to_element(town_input, recent_answer).perform()
    actions.send_keys(Keys.RETURN).perform()